In [1]:
import json
import os
import sys
import csv
import re
import base64

In [2]:
'''
#ChirpStack
ChirpStack_field_names = ['rawMeasurementId', 'timeIngested', 'publishedAt',
                          'devEUI', 'spreadingFactor', 'bandwidth', 'frequency', 'codeRate', 'alt', 'lat', 'lon']
ChirpStack_gtw_field_names = ['gatewayID', 'rssi', 'rxInfo.time', 'channel', 'loRaSNR', 'altitude', 'latitude', 'longitude']

#TTN
TTN_field_names = ['rawMeasurementId', 'timeIngested', 'time',
                   'hardware_serial', 'data_rate', 'frequency', 'coding_rate', 'alt', 'lat', 'lon']
TTN_gtw_field_names = ['gtw_id', 'rssi', 'gateways.time', 'channel', 'snr', 'altitude', 'latitude', 'longitude']

#TTS
TTS_field_names = ['rawMeasurementId', 'timeIngested', 'uplink_message.received_at',
                   'dev_eui', 'spreading_factor', 'bandwidth', 'frequency', 'coding_rate', 'alt', 'lat', 'lon']
TTS_gtw_field_names = ['gateway_ids', 'rssi', 'rx_metadata.time', 'channel_index', 'snr', 'altitude', 'latitude', 'longitude']
'''

#ChirpStack
ChirpStack_field_names = ['rawMeasurementId', 'timeIngested', 'publishedAt',
                          'devEUI', 'fCnt', 'spreadingFactor', 'bandwidth', 'frequency', 'codeRate']
ChirpStack_gtw_field_names = ['gatewayID', 'rssi', 'time', 'channel', 'loRaSNR']

#TTN
TTN_field_names = ['rawMeasurementId', 'timeIngested', 'time',
                   'hardware_serial', 'counter', 'data_rate', 'frequency', 'coding_rate']
TTN_gtw_field_names = ['gtw_id', 'rssi', 'gateways.time', 'channel', 'snr']

#TTS
TTS_field_names = ['rawMeasurementId', 'timeIngested', #uplink_message.received_at
                   'dev_eui', 'f_cnt', 'spreading_factor', 'bandwidth', 'frequency', 'coding_rate']
TTS_gtw_field_names = ['gateway_ids', 'rssi', 'rx_metadata.time', 'channel_index', 'snr']

# Data messages of different servers have different key names that have the same meaning
# TTN_TTS_ChirpStack is used to tranform the format of data messages of TTN and TTS to be the same as ChirpStack
TTN_TTS_ChirpStack = {
    # TTN -> ChirpStack
    # Value of 'hardware_serial' or 'devEUI' is EUI(Extended Unique Identifier) of a device
    'hardware_serial': 'devEUI',
    'SF': 'spreadingFactor',
    'BW': 'bandwidth',
    'coding_rate': 'codeRate',
    'gtw_id': 'gatewayID',
    'snr': 'loRaSNR',
    'time': 'publishedAt',
    'counter': 'fCnt',
    
    # TTS -> ChirpStack
    'dev_eui': 'devEUI',
    'spreading_factor': 'spreadingFactor',
    #'coding_rate': 'codeRate',
    'gateway_ids': 'gatewayID',
    #'snr': 'loRaSNR'
    #'received_at': 'publishedAt',
    'channel_index': 'channel',
    'f_cnt': 'fCnt'
}

# Field(Key) names that can be used to extract gateway information from the data messages of different servers
SERVER_GTW = {
    # server - keyname
    'ChirpStack': 'rxInfo',
    'TTN': 'gateways',
    'TTS': 'rx_metadata'
}
#TTN_TTS_gatewayIDs = []

TTN_TTS_gtwIDs = {
    'somsix-outdoor-1': 'AC1F09FFFE0465BF', 
    'utwente-rav-rooftop': 'A840411DA56C4150', 
    'utwente-lg308-03': 'A840411EADFC4150', 
    'loragw01-schoeppingen-reservebecken': '3133303714004D00', 
    #'000080029ccc08f3': '000080029CCC08F3', 
    'ttn-apeldoorn-de-mheen': '0031552048001A03', 
    'gateway-test-kantoor': 'A84041204E6C4150', 
    'ttn-apeldoorn-kadaster': '0031552048001A06', 
    'kerlink-awm-ut': '0000024B080301BF', 
    'utwente-enschede-macrocell': '1DEE039AAC75C307', 
    'irvine-gateway': '00005813D342B495', 
    'utwente-lg308-02': 'A840411EAE004150', 
    'ledlicht-engineering-lorawan-gateway': '58A0CBFFFE803D11'
}

In [3]:
# cvt: convert field names
# cvt_obj: TTN_TTS_ChirpStack
def extract_field_values(json_obj, field_names, server_name = 'ChirpStack', cvt_obj = None):
    field_values = {}
    
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            if key in field_names:
                # Process TTN and TTS because some of the field(key) names of their messages need to be converted to be the same as ChirpStack
                if server_name != 'ChirpStack':
                    if server_name == 'TTS' and key == 'time':
                            print(key)
                    # 'data_rate' in the message of TTN has a pattern like SF7BW125 
                    # which means Spreading Factor is 7 and Bandwidth is 125K.
                    if key == 'data_rate':
                        pattern = r'([A-Z]{2})(\d+)'
                        matches = re.findall(pattern, str(value))
                        for match in matches:
                            key = match[0]
                            value = int(match[1])
                            key = cvt_obj[key]
                            field_values[key] = value
                    elif key == 'gateway_ids':
                        if value['gateway_id'] == 'packetbroker':
                            if 'packet_broker' in json_obj and 'forwarder_gateway_eui' in json_obj['packet_broker']:
                                key = cvt_obj[key]
                                value = json_obj['packet_broker']['forwarder_gateway_eui']
                                field_values[key] = value
                            '''
                            if 'packet_broker' in json_obj and 'forwarder_gateway_eui' in json_obj['packet_broker']:
                                key = cvt_obj[key]
                                field_values[key] = json_obj['packet_broker']['forwarder_gateway_eui']
                            '''
                        else: 
                            if 'eui' in value:
                                key = cvt_obj[key]
                                field_values[key] = value['eui']
                                
                                '''
                                if value not in TTN_TTS_gatewayIDs:
                                    TTN_TTS_gatewayIDs.append(value)
                                '''

                                value = value['eui']
                            
                    # field(key) that needs to be converted
                    elif key in cvt_obj:
                        key = cvt_obj[key]
                        field_values[key] = value
                    # field(key) name that is the same as that of ChirpStack will not be converted
                    else:
                        field_values[key] = value
                # Procesee ChirpStack    
                else:
                    field_values[key] = value
                    
                if key == 'devEUI':
                    # If a device's EUI(Extended Unique Identifier) is base64 encoded, convert it to a 64-bit hex string
                    if len(value) == 12 and value.endswith('='):
                        field_values[key] = base64_to_hex(value).upper()
                elif key == 'gatewayID':
                    #If a gateway's EUI(Extended Unique Identifier) is base64 encoded, convert it to a 64-bit hex string
                    if len(value) == 12 and value.endswith('='):
                        field_values[key] = base64_to_hex(value).upper()
                    #If a gateway's EUI(Extended Unique Identifier) starts with 'eui-', remove 'eui-' from the gateway's EUI
                    elif value.startswith('eui-'):
                        field_values[key] = value.replace('eui-', '').upper()
                    elif value in TTN_TTS_gtwIDs:
                         field_values[key] = TTN_TTS_gtwIDs[value].upper()
                    elif any(c.islower() for c in value):
                        field_values[key] = value.upper()
                elif key == 'bandwidth':
                    # One example: bandwidth can be '125' or '125000' in the message but '125' and '125000 are actually the same
                    if value > 1000:
                        field_values[key] = int(value / 1000)
                    # One example: bandwidth can be '125' or '125000' in the message but '125' and '125000 are actually the same
                elif key == 'frequency':
                    # One example: frequency can be '867.9' or '867900000' in the message but '867.9' or '867900000' are actually the same here
                    if int(value) > 1000:
                        field_values[key] = int(value) / 1000000
                        
            # Do not convert 'time' in the gateway information
            # Extract 'time' from 'gateways'
            elif key == 'time' and (SERVER_GTW[server_name] + '.' + key) in field_names:
                field_values[key] = value
            elif isinstance(value, dict):
                if key == 'uplink_message' and 'received_at' in value:
                    field_values['publishedAt'] = value['received_at']
                # Since the json object is deeply nested, we may need to use recursion to get the value from a specific key
                nested_values = extract_field_values(value, field_names, server_name, cvt_obj)
                field_values.update(nested_values)
    '''
    elif isinstance(value, list):
        nested_values  = extract_field_values(value, field_names, cvt, cvt_obj)
        field_values.update(nested_values)
    '''

    return field_values

# Check if a string is a 64-bit hex string
def is_64bit_hex_string(string):
    pattern = r"^[0-9a-fA-F]{16}$"
    match = re.match(pattern, string)
    return match is not None

# convert base64 to 64-bit hex
def base64_to_hex(encoded_string):
    decoded_bytes = base64.b64decode(encoded_string)
    hex_string = decoded_bytes.hex()
    return hex_string

def flatten_info(csv_writer, data_obj, gtw_field_names, server_name = 'ChirpStack', cvt_obj = None):
    # Get the gateway information
    gateways_info = data_obj[SERVER_GTW[server_name]]
    
    # If the ID of a device is presented in the message and there is corresponding gateway information
    if data_obj['devEUI'] and gateways_info:
        # Delete the gateway information after extracting it
        del data_obj[SERVER_GTW[server_name]]
        # data_obj contains device information and some other information
        common_info = data_obj 
        common_info['networkServer'] = server_name
        # There can be multiple gateways receiving the message sent by a device
        for gateway in gateways_info:
            # Extract the information of a gateway
            gateway_info = extract_field_values(gateway, gtw_field_names, server_name, cvt_obj)
            
            '''
            if server_name == 'ChirpStack':
                uplinkID = gateway_info['uplinkID']
                del gateway_info['uplinkID']
                if uplinkID in uplinkIDs:
                    break
                else:
                    uplinkIDs.append(uplinkID)        
            '''
            
            # Add the gateway information
            common_info.update(gateway_info)
            #print(json.dumps(common_info, indent=4))

            csv_writer.writerow(common_info)

In [4]:
def to_csv(num):
    file_name = "data/loradata-" + str(num)
    #file_name = "data/test-data"
    json_file = file_name + '.json'
    out_file = file_name + ".csv"

    file_desc = open(json_file, 'r')

    # Header
    field_names = ['networkServer'] + ChirpStack_field_names + ChirpStack_gtw_field_names
    with open(out_file, 'w', newline='') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames = field_names)
        csv_writer.writeheader()

        # For each line:
        #
        # - Extract relevant fields for the message (device ID, ...)
        # - For each gateway in the metadata:
        #   - Write one line to a CSV that starts with the message fields, and then has the gateway fields
        #
        # Example:
        #
        # Message has device ID 'ff67', received on gateways 1, 2 and 3, you get three lines:
        #
        # device_id,gateway_id,gateway_timestamp,rssi,...
        # ff67,1,10:00h,-76
        # ff67,2,10:01h,-109
        # ff67,3,10:00h,-70
        for line in file_desc:
            line = line.strip() # strip CR LF and trailing white space

            if line.startswith('[') or line.startswith(']'): # line is open or close of the array
                continue

            if '"error":' in line:
                continue

            if line.endswith(','):
                line = line[:-1]

            try:
                obj = json.loads(line)

                # We don't know which server generated the data message at first
                # Assume it was generated by the server 'ChirpStack'
                server_name = 'ChirpStack'
                # Extract common field values and gateway information (still is a list)
                data_obj = extract_field_values(obj, ChirpStack_field_names + [SERVER_GTW[server_name]])
                # If it was indeed generated by the server 'ChirpStack'
                if 'devEUI' in data_obj:
                    if SERVER_GTW[server_name] in data_obj:
                        flatten_info(csv_writer, data_obj, ChirpStack_gtw_field_names)
                # If it wasn't generated by the server 'ChirpStack'
                else:
                    # And then we assume it was generated by the server 'TTN'
                    server_name = 'TTN'
                    data_obj = extract_field_values(obj, TTN_field_names + [SERVER_GTW[server_name]], server_name, cvt_obj = TTN_TTS_ChirpStack)
                    if 'devEUI' in data_obj:
                        if SERVER_GTW[server_name] in data_obj:
                            flatten_info(csv_writer, data_obj, TTN_gtw_field_names, server_name, cvt_obj = TTN_TTS_ChirpStack)
                    else:
                        # And then we assume it was generated by the server 'TTS'
                        server_name = 'TTS'
                        data_obj = extract_field_values(obj, TTS_field_names + [SERVER_GTW[server_name]], server_name, cvt_obj = TTN_TTS_ChirpStack)
                        if 'devEUI' in data_obj:
                            if SERVER_GTW[server_name] in data_obj:
                                flatten_info(csv_writer, data_obj, TTS_gtw_field_names, server_name, cvt_obj = TTN_TTS_ChirpStack) 
            except Exception as e:
                print('Exception:' + format(e))
    '''            
    # Specify the file path where you want to save the JSON file
    file_path = file_name + "-TTN_TTS_gatewayIDs.json"
    # Open the file in write mode
    with open(file_path, "w") as json_file:
        print(TTN_TTS_gatewayIDs)
        # Use the json.dump() function to write the list into the file
        json.dump(TTN_TTS_gatewayIDs, json_file)
    '''

In [ ]:
nums = [5, 11, 12, 13, 15, 17, 19, 21]
[to_csv(num) for num in nums]